In [1]:
import pandas as pd
import requests
import psycopg2
import os
import sqlalchemy
import json
import numpy as np

In [2]:
# Database credentials
ENDPOINT = 'datalaketwitter.c16p7bc6xhi2.us-east-1.rds.amazonaws.com'
DB_NAME = 'datalakeTwitter'
USERNAME = 'nadine'
PASSWORD = '30101995'

In [3]:
# connecting to the database
try:
    print("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    conn = psycopg2.connect("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
        
try:
    cur = conn.cursor()
    engine = sqlalchemy.create_engine("postgresql://{}:{}@{}:5432/{}".format(USERNAME,PASSWORD,ENDPOINT,DB_NAME))

    
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)
        
# Create sqlalchemy engine
engine = sqlalchemy.create_engine("postgresql://{}:{}@{}:5432/{}".format(USERNAME,PASSWORD,ENDPOINT,DB_NAME))

host=datalaketwitter.c16p7bc6xhi2.us-east-1.rds.amazonaws.com dbname=datalakeTwitter user=nadine password=30101995


In [4]:
conn.set_session(autocommit=True)

In [5]:
#fetching data from datalake and save it as variable
data_history = engine.execute('Select * FROM tasty_twitter_history_clean;').fetchall()

In [6]:
#fetching data from datalake and save it as variable
data = engine.execute('Select * FROM tasty_twitter_clean;').fetchall()

In [7]:
# putting the data into dataframes
df_history= pd.DataFrame(data_history)
df = pd.DataFrame(data)

In [8]:
#close the connections
conn.close()
#engine.close()

In [9]:
frames = [df,df_history]

In [10]:
# combining the data in 1 dataframe
df_full = df = pd.concat(frames, ignore_index=True)

In [11]:
# adapting the format of the date
df_full['created'] = pd.to_datetime(df_full.created)  

In [12]:
# order the df by date
df_full = df_full.sort_values(by = ['created']) 

In [13]:
#### loading df into other db
# Database credentials
username = 'warehouse_admin' 
password = 'LiNaZe2022' 
ipaddress = 'data-warehouse.cw7uns4wqebw.us-east-1.rds.amazonaws.com' 
port = '5432' 
dbname = 'datawarehouse'

In [14]:
# making a connection to the data warehouse
try:
    print("host={} dbname={} user={} password={}".format(ipaddress, dbname, username, password))
    conn = psycopg2.connect("host={} dbname={} user={} password={}".format(ipaddress, dbname, username, password))
    
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
        
try:
    cur = conn.cursor()
    engine = sqlalchemy.create_engine("postgresql://{}:{}@{}:5432/{}".format(username,password,ipaddress,dbname))

    
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)
        
# Create sqlalchemy engine
engine = sqlalchemy.create_engine("postgresql://{}:{}@{}:5432/{}".format(username,password,ipaddress,dbname))

host=data-warehouse.cw7uns4wqebw.us-east-1.rds.amazonaws.com dbname=datawarehouse user=warehouse_admin password=LiNaZe2022


In [16]:
#display(df_full)

In [ ]:
# creating the table if not exists
conn.set_session(autocommit=True)
cur.execute("CREATE TABLE IF NOT EXISTS tasty_twitter_clean (tweet TEXT, created DATE, favorites INT, count INT, url_short TEXT, displayed_url TEXT, expanded_url TEXT, symbols TEXT, recipe_link CHAR, recipe_name TEXT);")

In [ ]:
#adding the df into the database
df_full.to_sql('tasty_twitter_clean', engine, if_exists = 'replace', index = False)

In [17]:
#close the connections
conn.close()
#engine.close()